In [32]:
%matplotlib notebook
#import torch
#from spatial_scene_grammars.rules import ParentFrameGaussianOffsetRule, ParentFrameBinghamRotationRule
from pydrake.solvers.mathematicalprogram import MathematicalProgram
import numpy as np
import matplotlib.pyplot as plt
from pydrake.all import (
    Solve,
    SnoptSolver,
    MosekSolver,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    Polynomial
)

In [33]:
def AddRotationMatrixSpectrahedralSdpConstraint(prog, R):
    # Exact copy of this function in drake/solvers/rotation_constraint.c
    M = np.array([
        [1 - R[0, 0] - R[1, 1] + R[2, 2], R[0, 2] + R[2, 0], R[0, 1] - R[1, 0], R[1, 2] + R[2, 1]],
        [R[0, 2] + R[2, 0], 1 + R[0, 0] - R[1, 1] - R[2, 2], R[1, 2] - R[2, 1], R[0, 1] + R[1, 0]],
        [R[0, 1] - R[1, 0], R[1, 2] - R[2, 1], 1 + R[0, 0] + R[1, 1] + R[2, 2], R[2, 0] - R[0, 2]],
        [R[1, 2] + R[2, 1], R[0, 1] + R[1, 0], R[2, 0] - R[0, 2], 1 - R[0, 0] + R[1, 1] - R[2, 2]]
    ])
    prog.AddPositiveSemidefiniteConstraint(M)
    
def print_expr_info(e, name):
    is_polynomial = e.is_polynomial()
    if is_polynomial:
        print("%s is polynomial with degree %d" % (name, Polynomial(e).TotalDegree()))
    else:
        print("%s is not polynomial.")

def create_qqt_proxy_variables(prog, R):
    '''
    Given a rotation matrix R, create surrogate variables
    that equal the terms that appear in the outer product
    q * q^T, for quaternion q corresponding to R.
     
    We'll introduce intermediate variables for each term in that
    outer product, and constrain them to match the rotation matrix
    variables of the child node, which can be recovered from them
    via the standard quaternion-to-rotation-matrix conversion formula:

    R = [
        1 - 2yy - 2zz,   2xy - 2zw,      2xz + 2yw,
        2xy + 2zw,       1 - 2xx - 2zz,  2yz - 2xw,
        2xz - 2yw,       2yz + 2xw,      1 - 2xx - 2yy
    ]
    '''
    
    qqt_terms = prog.NewContinuousVariables(10)
    ww, wx, wy, wz, xx, xy, xz, yy, yz, zz = qqt_terms
    
    #w,x,y,z all in [-1, 1], so their products are as well.
    prog.AddBoundingBoxConstraint(-np.ones(10), np.ones(10), qqt_terms)
    # Build qqt matrix
    qqt = np.array([
        [ww, wx, wy, wz],
        [wx, xx, xy, xz],
        [wy, xy, yy, yz],
        [wz, xz, yz, zz]
    ])
    # The square terms are further obviously positive.
    prog.AddLinearConstraint(ww >= 0.)
    prog.AddLinearConstraint(xx >= 0.)
    prog.AddLinearConstraint(yy >= 0.)
    prog.AddLinearConstraint(zz >= 0.)
    # And the quaternion has unit norm.
    prog.AddLinearEqualityConstraint(ww + xx + yy + zz == 1.)

    # Enforce quaternion-bilinear-term-to-rotmat correspondence.
    prog.AddLinearEqualityConstraint(R[0, 0] == 1 - 2*yy - 2*zz)
    prog.AddLinearEqualityConstraint(R[0, 1] == 2*xy - 2*wz)
    prog.AddLinearEqualityConstraint(R[0, 2] == 2*xz + 2*wy)
    prog.AddLinearEqualityConstraint(R[1, 0] == 2*xy + 2*wz)
    prog.AddLinearEqualityConstraint(R[1, 1] == 1 - 2*xx - 2*zz)
    prog.AddLinearEqualityConstraint(R[1, 2] == 2*yz - 2*wx)
    prog.AddLinearEqualityConstraint(R[2, 0] == 2*xz - 2*wy)
    prog.AddLinearEqualityConstraint(R[2, 1] == 2*yz + 2*wx)
    prog.AddLinearEqualityConstraint(R[2, 2] == 1 - 2*xx - 2*yy)
    
    return qqt

# Set up a test SDP version the latent translation + rotation part of the parsing problem.

Suppose we have a hidden node that produces two observed objects at Parent-frame Gaussian and Bingham offsets. Set up an optimization inferring the latent node's pose if we're told the observed node poses.

In [34]:
# Poses of observed nodes.
observed_poses = [
    RigidTransform(p=[1., 0., 0.], rpy=RollPitchYaw(0., 0., 0.)),
    RigidTransform(p=[0., 1., 0.], rpy=RollPitchYaw(np.pi/2., 0., 0.))
]

# Translation relationship of hidden node to each observed node.
t_mean = np.zeros(3)
t_var = np.ones(3)


# Rotation relationship, as Bingham dist parameters.
# Orientation is a 4x4 orthogonal matrix; interpret the
# columns as quaternions, with the last column being the
# mode of the distribution.
R_orient = np.array([
    [0., 0., 0., 1],
    [1., 0., 0., 0.],
    [0., 1., 0., 0.],
    [0., 0., 1., 0.]
])
# Concentration for distribution around each of
# the "axes" (columns in R_orient). Negative, decreasing,
# and with last element 0 by convention.
R_conc = np.array([-10., -10., -10., 0.])
# (These parameters say that the child should be distributed
# relatively loosely around the orientation of the parent.)

# Level 1 of complexity: child poses are constants.

This has only one problematic bilinear term:
- $R_P^T * t_P$ (in the translation offset term)

In [35]:
prog = MathematicalProgram()

# Optimize for translation + rotation of latent node.
t_P = prog.NewContinuousVariables(3, "t")
R_P = prog.NewContinuousVariables(3, 3, "R")
AddRotationMatrixSpectrahedralSdpConstraint(prog, R_P)

    
# Add relative pose costs.
for child_pose in observed_poses:
    R_C = child_pose.rotation().matrix()
    t_C = child_pose.translation()
    
    # Parent frame Gaussian offset cost:
    # Calculate relative translation in parent frame ("t_PC")
    # and then penalize that quadratically following the Gaussian
    # mean and variance.
    precision = t_var
    t_PC = R_P.T.dot(t_C - t_P)
    err = t_PC - t_mean
    precision = np.linalg.inv(np.diag(t_var))
    gaussian_ll = -0.5 * err.T.dot(precision).dot(err)
    cost = prog.AddCost(gaussian_ll)
    
    print_expr_info(gaussian_ll, "Gaussian LL")
    
    # Parent from Bingham offset cost:
    # Calculate relative rotation in parent frame ("R_PC");
    # create a surrogate variable representing qq^T, and
    # penalize that in the Bingham log-prob cost.
    R_PC = R_P.T.dot(R_C)
    qqt = create_qqt_proxy_variables(prog, R_PC)
    bingham_ll = np.trace(np.diag(R_conc).dot(R_orient.T.dot(qqt.dot(R_orient))))
    cost = prog.AddCost(bingham_ll)
    
    print_expr_info(bingham_ll, "Bingham LL")

#solver = MosekSolver()
solver = SnoptSolver()
result = solver.Solve(prog)
#result = Solve(prog)
print(result.is_success())
print(result.GetSolution(t_P), result.GetSolution(R_P))

Gaussian LL is polynomial with degree 4
Bingham LL is polynomial with degree 1
Gaussian LL is polynomial with degree 4
Bingham LL is polynomial with degree 1


ValueError: SnoptSolver is unable to solve a MathematicalProgram with {ProgramAttributes: GenericCost, LinearCost, LinearConstraint, LinearEqualityConstraint, PositiveSemidefiniteConstraint}.

# Level 2 of complexity: child poses are also variables (which are used elsewhere in MI formulation to decide correspondence).

This cost isn't necessary to fix, but it significantly cleans up the formulation if we can. It involves the same rotation-times-translation bilinearity as before, but now with an additional rotation-times-rotation term (that feels more tractable / familiar to the SDP approach in e.g. TEASER).

This adds two new bilinear relationships:
- $R_P^T * t_C$ (in translation offset)
- $R_P^T * R_C$ (in rotation offset)

In [ ]:
prog = MathematicalProgram()

# Optimize for translation + rotation of latent node.
t_P = prog.NewContinuousVariables(3, "t")
R_P = prog.NewContinuousVariables(3, 3, "R")
AddRotationMatrixSpectrahedralSdpConstraint(prog, R_P)

    
# Add relative pose costs.
for k, child_pose in enumerate(observed_poses):
    R_C_obs = child_pose.rotation().matrix()
    t_C_obs = child_pose.translation()

    t_C = prog.NewContinuousVariables(3, "t_C%d" % k)
    R_C = prog.NewContinuousVariables(3, 3, "R_%d" % k)
    AddRotationMatrixSpectrahedralSdpConstraint(prog, R_P)
    prog.AddBoundingBoxConstraint(t_C_obs, t_C_obs, t_C)
    prog.AddBoundingBoxConstraint(R_C_obs.flatten(), R_C_obs.flatten(), R_C.flatten())
    
    
    # Parent frame Gaussian offset cost:
    # Calculate relative translation in parent frame ("t_PC")
    # and then penalize that quadratically following the Gaussian
    # mean and variance.
    precision = t_var
    t_PC = R_P.T.dot(t_C - t_P)
    err = t_PC - t_mean
    precision = np.linalg.inv(np.diag(t_var))
    gaussian_ll = -0.5 * err.T.dot(precision).dot(err)
    cost = prog.AddCost(gaussian_ll)
    
    print_expr_info(gaussian_ll, "Gaussian LL")
    
    # Parent from Bingham offset cost:
    # Calculate relative rotation in parent frame ("R_PC");
    # create a surrogate variable representing qq^T, and
    # penalize that in the Bingham log-prob cost.
    R_PC = R_P.T.dot(R_C)
    qqt = create_qqt_proxy_variables(prog, R_PC)
    bingham_ll = np.trace(np.diag(R_conc).dot(R_orient.T.dot(qqt.dot(R_orient))))
    cost = prog.AddCost(bingham_ll)
    
    print_expr_info(bingham_ll, "Bingham LL")

#solver = MosekSolver()
solver = SnoptSolver()
result = solver.Solve(prog)
#result = Solve(prog)
print(result.is_success())
print(result.GetSolution(t_P), result.GetSolution(R_P))